In [28]:
import sys

sys.path.append("../")

import numpy as np
import pandas_ta as ta
import joblib
import pandas as pd

from terminal.Extensions.MT5Ext import MT5Ext
from terminal.Extensions.Backtest import Backtest, SideType, Brick

In [29]:
# Carregar modelos
bought_model = joblib.load("../models/bought.joblib")
sold_model = joblib.load("../models/sold.joblib")

In [30]:
# Carregar os dados
ticks = np.load("./data/ticks_WIN$N_2024_7.npz", allow_pickle=True)["ticks"]
data = MT5Ext.create_ohlc_from_ticks(ticks, "20s")

del ticks

# Cálculo de indicadores
data["EMA_1"] = data.ta.ema(3)
data["EMA_2"] = data.ta.ema(8)
data["EMA_3"] = data.ta.ema(16)

data["EMA_1_CHANGE"] = data["EMA_1"].pct_change()
data["EMA_2_CHANGE"] = data["EMA_2"].pct_change()
data["EMA_3_CHANGE"] = data["EMA_3"].pct_change()

data.dropna(inplace=True)

# Pré-processamento de dados
data["CURRENT_CHANGE"] = data["close"].pct_change()
data["FORECAST_CHANGE"] = data["CURRENT_CHANGE"].shift(-1)

data.dropna(inplace=True)

# Criar características e rótulos
feature_columns = [
    "CURRENT_CHANGE"
]

In [ ]:
bought = data[data["close"] > data["open"]]
sold = data[data["close"] < data["open"]]

data["PRED"] = 0

data.loc[data["close"] > data["open"], "PRED"] = bought_model.predict(bought[feature_columns])
data.loc[data["close"] < data["open"], "PRED"] = sold_model.predict(sold[feature_columns])

In [ ]:
backtest = Backtest(slippage=0)

for i, d in data.iterrows():
    backtest.add_brick(Brick(date=i, close=d.close))

    signal = None
    
    if d.close > d.open: 
        signal = SideType.SELL if d.PRED == 0 else None
    elif d.close < d.open:
        signal = SideType.BUY if d.PRED == 0 else None

    if signal == SideType.BUY and (
        backtest.current_position is None
        or backtest.current_position.type == SideType.SELL
    ):
        if backtest.current_position is not None:
            backtest.close_position()
        backtest.add_position(SideType.BUY)
    elif signal == SideType.SELL and (
        backtest.current_position is None
        or backtest.current_position.type == SideType.BUY
    ):
        if backtest.current_position is not None:
            backtest.close_position()
        backtest.add_position(SideType.SELL)

In [ ]:
positions = pd.DataFrame(backtest.positions)
positions.to_excel("positions.xlsx")